In [8]:
from judger import Judger
import json
#import data
from importlib import reload
import os
import multiprocessing

#from predictor import Predictor

import pandas as pd
import importlib
import tensorflow as tf
import tensorflow_hub as hub
#importlib.reload(predictor)


## fun defination

In [92]:

# Calculate precision, recall and f1 value
# According to https://github.com/dice-group/gerbil/wiki/Precision,-Recall-and-F1-measure
def get_value(res):
    if res["TP"] == 0:
        if res["FP"] == 0 and res["FN"] == 0:
            precision = 1.0
            recall = 1.0
            f1 = 1.0
        else:
            precision = 0.0
            recall = 0.0
            f1 = 0.0
    else:
        precision = 1.0 * res["TP"] / (res["TP"] + res["FP"])
        recall = 1.0 * res["TP"] / (res["TP"] + res["FN"])
        f1 = 2 * precision * recall / (precision + recall)

    return precision, recall, f1

# Generate score for the first two subtasks
def gen_score(arr):
    sumf = 0
    y = {"TP": 0, "FP": 0, "FN": 0, "TN": 0}
    for x in arr:
        p, r, f = get_value(x)
        sumf += f
        for z in x.keys():
            y[z] += x[z]

    _, __, f_ = get_value(y)

    return (f_ + sumf * 1.0 / len(arr)) / 2.0

# Generatue all scores
def get_score(self, result):
    s1 = self.gen_score(result[0])
    s2 = self.gen_score(result[1])
    s3 = 1.0 * result[2]["score"] / result[2]["cnt"]
    return [s1, s2, s3]

def get_accusation_num(df_accu, accusation):
    keys = df_accu.accusation.tolist()
    keys = [x.strip() for x in keys]
    values = df_accu.accusation_num.tolist()
    dict_accu = dict(zip(keys, values))

    accusation_num = list()
    for accu in accusation:
        accusation_num.append([dict_accu.get(x.replace("'", "").strip()) for x in accu.split(',')])
    return accusation_num

## prepare validation data

In [12]:
!pwd

/home/aida-zw/Desktop/ws/notebook


In [37]:
accusation_path = "../Data/sample/accu.txt"
law_path="../Data/sample/law.txt"
result_path = '../output/result.csv'

accu_dic = {}

f = open(accusation_path, "r")
task1_cnt = 0
for line in f:
    task1_cnt += 1
    accu_dic[line[:-1]] = task1_cnt

law_dic = {}
f = open(law_path, "r")
task2_cnt = 0
for line in f:
    task2_cnt += 1
    law_dic[int(line[:-1])] = task2_cnt
task1_cnt, task2_cnt

cnt = 0
result = [[], [], {}]
for a in range(0, task1_cnt):
    result[0].append({"TP": 0, "FP": 0, "TN": 0, "FN": 0})
for a in range(0, task2_cnt):
    result[1].append({"TP": 0, "FP": 0, "TN": 0, "FN": 0})
result[2] = {"cnt": 0, "score": 0}

df_result = pd.read_csv(result_path)

df_accu = pd.read_csv('../Data/sample/accu.csv')
df_result['pred_num'] = get_accusation_num(df_accu, df_result['prediction'])
df_result['gt_num'] = get_accusation_num(df_accu, df_result['truth'])

In [83]:
for index, row in df_result.iterrows():
    prediction = row['pred_num']
    ground_truth = row['gt_num']
    
    s1 = set(prediction)
    s2 = set(ground_truth)

    for a in range(0, task1_cnt):
        in1 = (a + 1) in s1
        in2 = (a + 1) in s2
        if in1:
            if in2:
                result[0][a]["TP"] += 1
            else:
                result[0][a]["FP"] += 1
        else:
            if in2:
                result[0][a]["FN"] += 1
            else:
                result[0][a]["TN"] += 1